## 1、simpletransformers

使用bert获取子模型分数

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install simpletransformers

     |████████████████████████████████| 225kB 35.1MB/s 
     |████████████████████████████████| 3.3MB 50.6MB/s 
     |████████████████████████████████| 245kB 51.0MB/s 
     |████████████████████████████████| 7.8MB 49.5MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 133kB 54.4MB/s 
     |████████████████████████████████| 1.8MB 45.3MB/s 
     |████████████████████████████████| 2.5MB 52.3MB/s 
     |████████████████████████████████| 1.2MB 45.8MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 245kB 59.5MB/s 
     |████████████████████████████████| 122kB 57.7MB/s 
     |████████████████████████████████| 81kB 13.2MB/s 
     |████████████████████████████████| 174kB 57.9MB/s 
     |████████████████████████████████| 112kB 58.7MB/s 
     |████████████████████████████████| 4.2MB 45.6MB/s 
     |████████████████████████████████| 133kB 55.6MB/s 
     |████████████████████████████████| 102kB 14.5MB

### 1.1 classification

In [ ]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
train = pd.read_csv('./drive/MyDrive/data/train.csv')
test = pd.read_csv('./drive/MyDrive/data/test.csv')

train['content'].fillna('', inplace=True)
test['content'].fillna('', inplace=True)

In [ ]:
import torch
cuda_available = torch.cuda.is_available()

oof = []
prediction = test[['id']]
prediction['bert_pred'] = 0

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train, train['label'])):
    train_df = train.iloc[trn_idx][['content', 'label']]
    valid_df = train.iloc[val_idx][['content', 'label']]
    train_df.columns = ['text', 'labels']
    valid_df.columns = ['text', 'labels']

    def get_model_args(dir):
      model_args = ClassificationArgs()
      model_args.max_seq_length = 128
      model_args.train_batch_size = 16
      model_args.num_train_epochs = 2
      model_args.sliding_window=True
      model_args.evaluate_during_training = True
      model_args.evaluate_during_training_verbose = True
      model_args.fp16 = False
      model_args.no_save = True
      model_args.save_steps = -1
      model_args.overwrite_output_dir = True
      model_args.output_dir = dir
      return model_args
    
    model_args = get_model_args('./drive/MyDrive/models/bert_fold_' + str(fold_id))
    model = ClassificationModel('bert',
                   'hfl/chinese-roberta-wwm-ext',    # community models: hfl/chinese-electra-180g-base-discriminator
                   use_cuda=cuda_available,
                   args=model_args)
    model.train_model(train_df, eval_df=valid_df)
    _, vaild_outputs, _ = model.eval_model(valid_df)
    
    df_oof = train.iloc[val_idx][['id', 'label']].copy()
    df_oof['bert_pred'] = [np.mean(output, axis=0)[1] for output in vaild_outputs]
    oof.append(df_oof)
    
    _, test_outputs = model.predict([text for text in test['content']])
    prediction['bert_pred'] += np.array([np.mean(output, axis=0)[1] for output in test_outputs]) / kfold.n_splits
    
    del model, train_df, valid_df, vaild_outputs, test_outputs
    gc.collect()

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

...

Running Epoch 3 of 4:   0%|          | 0/601 [00:00<?, ?it/s]

  0%|          | 0/2400 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/18000 [00:00<?, ?it/s]

  0%|          | 0/2251 [00:00<?, ?it/s]

In [ ]:
df_oof = pd.concat(oof)
df_oof = df_oof.sort_values(by='id')
df_oof.head(10)

In [ ]:
prediction.head()

,id,bert_pred
0,0,-4.662366
1,1,-4.102187
2,2,4.132143
3,3,-4.597604
4,4,-4.657640


In [ ]:
i_bst = 0
bst = 0
for i in np.arange(df_oof.bert_pred.min(), df_oof.bert_pred.max(), 0.1):
    df_oof['pred_label'] = df_oof['bert_pred'].apply(lambda x: 1 if x >= i else 0)
    score = f1_score(df_oof['label'], df_oof['pred_label'])
    # print(i, 'f1_score:', score)
    if score> bst:
        i_bst = i
        bst = score
print(i_bst, 'f1_score:', bst)
# 0.9234375

-0.16788825988771272 f1_score: 0.9173811379579112


In [ ]:
prediction['label'] = prediction['bert_pred'].apply(lambda x: 1 if x >= i_bst else 0)
prediction['label'].value_counts()

0    16122
1     1878
Name: label, dtype: int64

In [ ]:
df_oof[['id', 'bert_pred']].to_csv('./drive/MyDrive/output/roberta_pred_oof2.csv', index=False)
prediction[['id', 'bert_pred']].to_csv('./drive/MyDrive/output/roberta_pred_test2.csv', index=False)

使用保存的模型预测

In [ ]:
testB = pd.read_csv('./drive/MyDrive/data/test.csv')
testB['content'].fillna('', inplace=True)
predictionB = test_b[['id']]
predictionB['bert_pred'] = 0

model2 = ClassificationModel('bert', './drive/MyDrive/outputs/bert_fold_0/')
_, test_outputs = model.predict([text for text in testB['content']])
predictionB['bert_pred'] += np.array([np.mean(output, axis=0)[1] for output in test_outputs]) / kfold.n_splits

### 1.2 sentence-pair classification

In [ ]:
import torch
cuda_available = torch.cuda.is_available()

oof = []
prediction = test[['id']]
prediction['bert_pred'] = 0

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train, train['label'])):
    train_df = train.iloc[trn_idx][['level_4', 'content', 'label']]
    valid_df = train.iloc[val_idx][['level_4', 'content', 'label']]
    train_df.columns = ['text_a', 'text_b', 'labels']
    valid_df.columns = ['text_a', 'text_b', 'labels']
    
    def get_model_args(dir):
        model_args = ClassificationArgs()
        model_args.max_seq_length = 256
        model_args.train_batch_size = 16
        model_args.num_train_epochs = 3
        model_args.fp16 = False
        model_args.no_save = True
        model_args.overwrite_output_dir = True
        model_args.output_dir = dir
        return model_args

    model_args = get_model_args('./drive/MyDrive/models/bert_pair_fold_' + str(fold_id))   
    model = ClassificationModel('bert',
                  'hfl/chinese-roberta-wwm-ext',
                  num_labels=2,
                  use_cuda=True,
                  cuda_device=0,
                  args=model_args)
    model.train_model(train_df, eval_df=valid_df)

    _, vaild_outputs, _ = model.eval_model(valid_df)
    
    df_oof = train.iloc[val_idx][['id', 'label']].copy()
    df_oof['bert_pred'] = vaild_outputs[:,1]
    oof.append(df_oof)
    _, test_outputs = model.predict([list(text) for text in test[['level_4', 'content']].values])
    prediction['bert_pred'] += test_outputs[:, 1] / kfold.n_splits
    
    del model, train_df, valid_df, vaild_outputs, test_outputs
    gc.collect()

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/600 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/600 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/600 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/600 [00:00<?, ?it/s]

In [ ]:
df_oof = pd.concat(oof)
df_oof = df_oof.sort_values(by='id')
df_oof.head()

,id,label,bert_pred
0,0,0,-2.932117
1,1,1,3.180629
2,2,0,0.524308
3,3,0,-4.449299
4,4,0,-4.220698


In [ ]:
prediction.head()

,id,bert_pred
0,0,-4.426371
1,1,-4.116755
2,2,3.641199
3,3,-4.368435
4,4,-4.412270


In [ ]:
i_bst = 0
bst = 0
for i in np.arange(df_oof.bert_pred.min(), df_oof.bert_pred.max(), 0.1):
    df_oof['pred_label'] = df_oof['bert_pred'].apply(lambda x: 1 if x >= i else 0)
    score = f1_score(df_oof['label'], df_oof['pred_label'])
    # print(i, 'f1_score:', score)
    if score> bst:
        i_bst = i
        bst = score
print(i_bst, 'f1_score:', bst)  # 0.9200316706254948

0.7310832977294712 f1_score: 0.9200316706254948


In [ ]:
df_oof[['id', 'bert_pred']].to_csv('./drive/MyDrive/output/roberta2_pred_oof3.csv', index=False)
prediction[['id', 'bert_pred']].to_csv('./drive/MyDrive/output/roberta2_pred_test3.csv', index=False)

## 2、sentence-transformers

获取文本相关性

- 直接使用预训练模型，获取文本相关性
- 使用训练样本微调之后，获取文本相关性

In [ ]:
pip install -U sentence-transformers

In [ ]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distiluse-base-multilingual-cased')
# distiluse-base-multilingual-cased 蒸馏得到的，官方预训练好的模型

In [ ]:
import pandas as pd

train = pd.read_csv('./drive/MyDrive/data/train.csv').iloc[:,4:]
test = pd.read_csv('./drive/MyDrive/data/test.csv').iloc[:,4:]

train['content'].fillna('', inplace=True)
test['content'].fillna('', inplace=True)

In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset 
from sentence_transformers import InputExample, evaluation, losses
from torch.utils.data import DataLoader

# Define your train examples.
train_datas = []
for i in train_data:
    train_datas.append(InputExample(texts=[i[0], i[1]], label=float(i[2])))

# Define your evaluation examples
sentences1,sentences2,scores = [],[],[]
for i in valid_data:
    sentences1.append(i[0])
    sentences2.append(i[1])
    scores.append(float(i[2]))

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)


# Define your train dataset, the dataloader and the train loss
train_dataset = SentencesDataset(train_datas, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

# Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, 
          evaluator=evaluator, evaluation_steps=200, output_path='./drive/MyDrive/two_albert_similarity_model')

In [ ]:
model = SentenceTransformer('./two_albert_similarity_model')

In [ ]:
def get_embedding(data):
    sentences1, sentences2 = [], []
    for i in data:
        sentences1.append(i[0])
        sentences2.append(i[1])

    # Sentences are encoded by calling model.encode()
    emb1 = model.encode(sentences1, convert_to_tensor=True)
    emb2 = model.encode(sentences2, convert_to_tensor=True)
    cos_sim = torch.cosine_similarity(emb1, emb2, dim=1)

    return emb1, emb2, cos_sim

In [ ]:
tr_emb1, tr_emb2, tr_cos_sim = get_embedding(train_all_data)
ts_emb1, ts_emb2, ts_cos_sim = get_embedding(test_data)

emb1_cat = np.concatenate((tr_emb1, ts_emb1), axis=0)
emb2_cat = np.concatenate((tr_emb2, ts_emb2), axis=0)

In [ ]:
# 保存embedding和cosine-similarity
from sklearn.decomposition import TruncatedSVD

df_tr = tr_data[['id']].copy()
df_ts = ts_data[['id']].copy()
n_components = 16
svd = TruncatedSVD(n_components=n_components)

svd.fit(emb1_cat)
emb1_cat_svd = svd.transform(emb1_cat)
emb1_tr_svd = emb1_cat_svd[:tr_data.shape[0], :]
emb1_ts_svd = emb1_cat_svd[tr_data.shape[0]:, :]

for i in range(n_components):
    df_tr[f'l4_emb_{i}'] = emb1_tr_svd[:, i]
    df_ts[f'l4_emb_{i}'] = emb1_ts_svd[:, i]

svd.fit(emb2_cat)
emb2_cat_svd = svd.transform(emb2_cat)
emb2_tr_svd = emb2_cat_svd[:tr_data.shape[0], :]
emb2_ts_svd = emb2_cat_svd[tr_data.shape[0]:, :]

for i in range(n_components):
    df_tr[f'ct_emb_{i}'] = emb2_tr_svd[:, i]
    df_ts[f'ct_emb_{i}'] = emb2_ts_svd[:, i]

df_tr['cos_sim'] = tr_cos_sim
df_ts['cos_sim'] = ts_cos_sim

In [ ]:
df_tr.to_csv('./drive/MyDrive/data/train_svd_sim.csv', index=False)
df_ts.to_csv('./drive/MyDrive/data/test_svd_sim.csv', index=False)